<a href="https://colab.research.google.com/github/Elabs-llc/Elabs-llc/blob/main/VPN_Server_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# vpn_server.py
# This is a simplified proof-of-concept VPN server.
# WARNING: This is for educational purposes ONLY and is NOT secure.

import socket
import threading
import os
from cryptography.fernet import Fernet

# --- Configuration ---
HOST = '0.0.0.0'  # Listen on all network interfaces
PORT = 8888
KEY = b'' # Will be loaded from a file

# --- Load Encryption Key ---
def load_key():
    """
    Loads the encryption key from a file named 'secret.key'.
    If the file doesn't exist, it generates a new key and saves it.
    """
    global KEY
    if not os.path.exists('secret.key'):
        print("Key not found. Generating a new one...")
        key = Fernet.generate_key()
        with open('secret.key', 'wb') as key_file:
            key_file.write(key)
        print("New key generated and saved to 'secret.key'")

    with open('secret.key', 'rb') as key_file:
        KEY = key_file.read()
    print("Encryption key loaded successfully.")

# --- Main Client Handling Logic ---
def handle_client(client_socket, addr, fernet):
    """
    Handles a single client connection.
    Receives encrypted data, decrypts it, forwards it,
    receives the response, encrypts it, and sends it back.
    """
    print(f"[NEW CONNECTION] {addr} connected.")
    try:
        while True:
            # Receive encrypted data from the client
            encrypted_data = client_socket.recv(4096)
            if not encrypted_data:
                break

            # Decrypt the data
            try:
                decrypted_data = fernet.decrypt(encrypted_data)
                # The decrypted data should be a simple HTTP request for this example
                print(f"[{addr}] Decrypted request: {decrypted_data.decode('utf-8', 'ignore')[:100]}...")

                # --- This is the core "forwarding" part ---
                # A real VPN would parse this request and forward it appropriately.
                # For this simple example, we'll connect to a fixed target (google.com)
                # and forward the raw request.

                # WARNING: A real VPN would need to handle DNS, different protocols, etc.
                # This is a huge simplification.
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as forward_socket:
                    # For simplicity, we connect to Google on port 80 (HTTP)
                    forward_socket.connect(('www.google.com', 80))
                    forward_socket.sendall(decrypted_data)

                    # Get the response from the target server
                    response_data = forward_socket.recv(8192) # Increased buffer for web pages

                    # Encrypt the response and send it back to the client
                    encrypted_response = fernet.encrypt(response_data)
                    client_socket.sendall(encrypted_response)

            except Exception as e:
                print(f"[ERROR] Decryption or forwarding failed for {addr}: {e}")
                break

    except Exception as e:
        print(f"[ERROR] An error occurred with client {addr}: {e}")
    finally:
        print(f"[CONNECTION CLOSED] {addr}")
        client_socket.close()

# --- Main Server Function ---
def main():
    """
    Initializes the server and listens for incoming connections.
    """
    load_key()
    fernet = Fernet(KEY)

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen(5)
    print(f"[*] Server listening on {HOST}:{PORT}")

    try:
        while True:
            client_socket, addr = server.accept()
            # Create a new thread for each client to handle them concurrently
            client_handler = threading.Thread(target=handle_client, args=(client_socket, addr, fernet))
            client_handler.start()
    except KeyboardInterrupt:
        print("\n[*] Server is shutting down.")
    finally:
        server.close()

if __name__ == "__main__":
    main()